In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("data/practice/task2.csv", sep=";", 
                     names = ['index', 'text', 'cause', 'effect', 'offset_sentence2',
                              'offset_sentence3', 'cause_start', 'cause_end', 
                              'effect_start', 'effect_end', 'sentence'],
                    header=0)
                    

In [12]:
train.iloc[0]["sentence"]

" <e2>The Sunshine State drew in a net influx of about $17.7 billion in adjusted gross income (AGI)  -  most of which (72 percent) came from those aged 55 and older.</e2> <e1>It is consistently one of the most popular destinations for retirees due to affordability and low taxes.</e1> Florida's $17.7 billion in net AGI dwarves the remaining 19 states that saw a positive net influx of income  -  which combined for a total of $19.4 billion."

In [14]:
train.iloc[0]["cause"]

' It is consistently one of the most popular destinations for retirees due to affordability and low taxes.'

In [15]:
train.iloc[0]["effect"]

' The Sunshine State drew in a net influx of about $17.7 billion in adjusted gross income (AGI)  -  most of which (72 percent) came from those aged 55 and older.'

In [3]:
train.iloc[0]

index                                                    0001.00005.1
text                 The Sunshine State drew in a net influx of ab...
cause                It is consistently one of the most popular de...
effect               The Sunshine State drew in a net influx of ab...
offset_sentence2                                                  159
offset_sentence3                                                  264
cause_start                                                       160
cause_end                                                         264
effect_start                                                        0
effect_end                                                        159
sentence             <e2>The Sunshine State drew in a net influx o...
Name: 0, dtype: object

In [ ]:
"""
Tagging scheme: 
    C-B
    C-I
    E-B
    E-I
"""

